In [6]:
pip install imutils

In [7]:
# Necessary imports
import cv2
import dlib
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import imutils

In [8]:
#Initialize color [color_type] = (Blue, Green, Red)
color_blue = (254,207,110)
color_cyan = (255,200,0)
color_black = (0, 0, 0)

# Use input () function to capture from user requirements for mask type and mask colour
choice1 = input("Please select the choice of mask color\nEnter 1 for blue\nEnter 2 for black:\n")
choice1 = int(choice1)

if choice1 == 1:
    choice1 = color_blue
    print('You selected mask color = blue')
elif choice1 == 2:
    choice1 = color_black
    print('You selected mask color = black')
else:
    print("invalid selection, please select again.")
    input("Please select the choice of mask color\nEnter 1 for blue\nEnter 2 for black :\n")


choice2 = input("Please enter choice of mask type coverage \nEnter 1 for high \nEnter 2 for medium \nEnter 3 for low :\n")
choice2 = int(choice2)

if choice2 == 1:
    # choice2 = fmask_a
    print(f'You chosen wide, high coverage mask')
elif choice2 == 2:
    # choice2 = fmask_c
    print(f'You chosen wide, medium coverage mask')
elif choice2 == 3:
    # choice2 = fmask_e
    print(f'You chosen wide, low coverage mask')
else:
    print("invalid selection, please select again.")
    input("Please enter choice of mask type coverage \nEnter 1 for high \nEnter 2 for medium \nEnter 3 for low :\n")

# print(choice2)



In [9]:
p = "../input/shape-predictor/shape_predictor_68_face_landmarks.dat"
def function(path):
  img= cv2.imread(path)
  img2 = imutils.resize(img, width = 500)
  gray=cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

  # Initialize dlib's face detector
  detector = dlib.get_frontal_face_detector()



  faces = detector(gray, 1)

  # printing the coordinates of the bounding rectangles
  #print(faces)
  #print("Number of faces detected: ", len(faces))


  # Path of file

  # Initialize dlib's shape predictor
  predictor = dlib.shape_predictor(p)

  # Get the shape using the predictor

  for face in faces:
      landmarks = predictor(gray, face)

      points = []
      for i in range(1, 16):
          point = [landmarks.part(i).x, landmarks.part(i).y]
          points.append(point)
      # print(points)

      # Coordinates for the additional 3 points for wide, high coverage mask - in sequence
      mask_a = [((landmarks.part(42).x), (landmarks.part(15).y)),
                ((landmarks.part(27).x), (landmarks.part(27).y)),
                ((landmarks.part(39).x), (landmarks.part(1).y))]

      # Coordinates for the additional point for wide, medium coverage mask - in sequence
      mask_c = [((landmarks.part(29).x), (landmarks.part(29).y))]

      # Coordinates for the additional 5 points for wide, low coverage mask (lower nose points) - in sequence
      mask_e = [((landmarks.part(35).x), (landmarks.part(35).y)),
                ((landmarks.part(34).x), (landmarks.part(34).y)),
                ((landmarks.part(33).x), (landmarks.part(33).y)),
                ((landmarks.part(32).x), (landmarks.part(32).y)),
                ((landmarks.part(31).x), (landmarks.part(31).y))]

      fmask_a = points + mask_a
      fmask_c = points + mask_c
      fmask_e = points + mask_e

      # mask_type = {1: fmask_a, 2: fmask_c, 3: fmask_e}
      # mask_type[choice2]

      # Using Python OpenCV – cv2.polylines() method to draw mask outline for [mask_type]:
      # fmask_a = wide, high coverage mask,
      # fmask_c = wide, medium coverage mask,
      # fmask_e  = wide, low coverage mask

      fmask_a = np.array(fmask_a, dtype=np.int32)
      fmask_c = np.array(fmask_c, dtype=np.int32)
      fmask_e = np.array(fmask_e, dtype=np.int32)

      mask_type = {1: fmask_a, 2: fmask_c, 3: fmask_e}
      mask_type[choice2]

      # change parameter [mask_type] and color_type for various combination
      img3 = cv2.polylines(img2, [mask_type[choice2]], True, choice1, thickness=2, lineType=cv2.LINE_8)

      # Using Python OpenCV – cv2.fillPoly() method to fill mask
      # change parameter [mask_type] and color_type for various combination
      img4 = cv2.fillPoly(img3, [mask_type[choice2]], choice1, lineType=cv2.LINE_AA)
      img5=cv2.resize(img4, (48, 48))
      return img5


In [10]:
file_dir='../input/fer2013happy/happy'
n=0
for root, dirs, files in  os.walk(file_dir):
    for i in files:
        n=n+1
print(n)

In [11]:
label = np.zeros([n], dtype=int)
data = np.zeros([n, 48*48*3], dtype=np.uint8)

In [ ]:
file_dir='../input/fer2013happy/happy'
t=1
i=0
j=0
for root, dirs, files in  os.walk(file_dir):
    for i in range(0,n):
        t=files[i]
        if t.endswith('.jpg'):
            tmp_path = os.path.join(file_dir, t)
            img6=function(tmp_path)
            if img6 is not None:
               label[j] = 3
               data[j][0:48*48*3] = np.ndarray.flatten(img6)
               j=j+1
            i = i + 1
           
           


In [ ]:
import csv
with open(r"./face.csv","w") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['emotion', 'pixels'])
    for i in range(len(label)):
        data_list = list(data[i])
        b = " ".join(str(x) for x in data_list)
        l = np.hstack([label[i], b])
        writer.writerow(l)


In [ ]:

import pandas as pd
data = pd.read_csv(r'./face.csv', dtype='a')
label = np.array(data['emotion'])
img_data = np.array(data['pixels'])
for i in range(3):
    x = img_data[i]
    x = np.fromstring(x, dtype=float, sep=' ')
    x = x/x.max()
    img_x = np.reshape(x, (48, 48,3))
    plt.subplot(3,3,i+1)
    plt.axis('off')

    plt.imshow(img_x, plt.cm.gray)
plt.show()
